# LAB 1: Making Propositional Logic (PL) and Querying PL Knowledge base
In this lab you will implement propositional logic which will be the foundation of our AI agent.
Initially it might seem strange to implement propositional logic, since python already has it implemented in the simple ```not```, ```and```, ```or```, and ```==``` operators (implication can be derived from these as well).
While its true that any propositional logic statement can be evaluated using Python, these statements cannot be stored in their unevaluated form (which is desirable to form a Knowledge Base- KB).
Additionally, we cannot create variables without assigning them value in standard Python, but when we later want to iterate over all possible models the variables will need to take on many possible values.

So we will need to implement our own version of propositional logic in Python.
For variables we will use strings to represent them and later we can use dictionaries to represent models and then look up the value of each variable in that model.
For True and False we will simply use the existing ```True``` and ```False``` values in Python.
For all the operators we will implement classes that contain the logic sentences they operate on.

Task 1:
 Given  a skeleton for the "and" operator, you need to expand it to each of the other operators.


Task 2:
Add an option, which allows to deal with repeated use of the same operation over multiple sentences.

Task 3:
Implement a function that given a binary operator and a list of sentences chains them togehter (e.g. ```fun(And, [s1, s2, s3]) -> And(S1, And(s2, s3))```).


In [117]:
class And:
    def __init__(self, *args):
        #self.A = A
        #self.B = B
        if len(args) < 2:
            raise ValueError("And operator needs at least two operands")
        self.operands = list(args)

    def __repr__(self):
        return f"({' ∧ '.join(map(str, self.operands))})"
        #return f"({self.A} ∧ {self.B})"

class Or:
    def __init__(self, *args):
        #self.A = A
        #self.B = B
        if len(args) < 2:
            raise ValueError("OR operator needs at least two operands")
        self.operands = list(args)
    def __repr__(self):
        return f"({' ∨ '.join(map(str, self.operands))})"
        #return f"({self.A} ∨ {self.B})"


class Equals:
     def __init__(self, A, B):
        self.A = A
        self.B = B
     def __repr__(self):
         return f"({self.A} ↔ {self.B})"

class Implies:
     def __init__(self, A, B):
        self.A = A
        self.B = B

     def __repr__(self):
        return f"({self.A} → {self.B})"

class Not:
     def __init__(self, A):
        self.A = A

     def __repr__(self):
        return f"¬{self.A}"

def full_sentence_data(operator_class, sentences):
    if not sentences:
        return None
    result = sentences[-1]
    for s in reversed(sentences[:-1]):
        result = operator_class(s, result)
    return result

## Evaluating propositional sentences in a given model

Being able to form sentences in propositional logic is not very useful if we cannot actually evaluate them. A start of the function can be found below, but it is missing most things.

Task 4:
Implement a fuction ```evaluate``` that takes a sentence in propositional logic and a dictonary with the truth values of the different variables in a given model and returns the truth value of the sentence in that model.

Note that he function for replacing the variables with their correct truth-value will currently give an error if a variable is missing, but we want it to return ```None``` to represent an unknown value instead.

Task 5:
Add a new thing to the mix to make evaluation of large models more efficient down the line.
The thing we will add is the concept of an unknown or uninteresting truth value using the Python ```None```-type.
The idea with unknown values is that instead of iterating over every value for every variable to evaluate a model, we can decide to only iterate over some variables and set the others as unknown.
However if an important variable is set to unknown we might not be able to get the correct answer.
In order to protect against this the ```evaluate``` function will return ```True``` if the sentence is true, ```False``` if its false, and ```None``` if we cannot know since some variables are unknown to us.
In order to deal with the ```None``` values you will have to think what the answer to many different operators should be. For example, we know that ```And(False, None)``` should be ```False``` since ```And``` always returns ```False``` when at least one sub-sentence is ```False```.
You will also need to experiment with Python to see if the Python operators give the correct behavior. Does ```False == None``` return ```None```.
If not, how to deal with it.




In [118]:
def evaluate(sentence, variables):
    '''
    Evaluates the value of a logic sentences using the values given to the variables in a dictionary.
    Args:
        sentence (any): A boolean, None, a variable string, or an operator class containing subsentences
        variables (dict): A dictionary mapping variables represented by strings to boolean values
    Returns (bool/None): The truth value of the sentence with the given variables or None if the answer is unknown
    '''
    if isinstance(sentence, bool):
        return sentence
    # If the sentence is a string, look up its truth value and return it
    if isinstance(sentence, str):
        return variables.get(sentence, None)
    # If the sentence is an And-operator, look up the truth value of its subsentences then use "and" on the results
    elif isinstance(sentence, And):
       ''' a = evaluate(sentence.A,variables)
        b = evaluate(sentence.B,variables)
        if a is None:
         return None
        return a and b
         result = True
         '''
       result = True
       unknown_data = False
       for op in sentence.operands:
            sub_sentence = evaluate(op, variables)
            if sub_sentence is False:
             return False
            elif sub_sentence is None:
              unknown_data = True
       return None if unknown_data else True

    elif isinstance(sentence, Or):
        '''
        a = evaluate(sentence.A,variables)
        b = evaluate(sentence.B,variables)
        return a or b
        '''
        result = False
        unknown_data = False
        for op in sentence.operands:
              val = evaluate(op, variables)
              if val is True:
                 return True
              elif val is None:
                 unknown_data = True

        return None if unknown_data else False

    elif isinstance(sentence, Equals):
        first = evaluate(sentence.A,variables)
        second = evaluate(sentence.B,variables)
        if first is None or second is None:
            return None
        return first == second

    elif isinstance(sentence, Implies):
        first = evaluate(sentence.A,variables)
        second = evaluate(sentence.B,variables)
        if first is False:
            return True
        if first is True and second is False:
            return False
        if first is True and second is None:
            return None
        if first is True:
            return True
        if first is None and second is True:
            return True
        return None

    elif isinstance(sentence, Not):
        fullsentence = evaluate(sentence.A,variables)
        if fullsentence is None:
            return None
        return not fullsentence
    else:
        raise TypeError(f"Unknown sentence type: {sentence}")


### Testing the evaluate function

Task 6: We have now created a propositional logic grammar and a function to evaluate the truth value of sentences for a given model.
In order to make sure everything is correct we introduce a testing space below and ask you to implement and evaluate a number of sentences for a given model in order to see that you get the correct result.

Below you are given a model and are then supposed to implement and run evaluation on the sentences below, with the given expected results:

#### Example 1 (Group 1-5):
- $rain \leftrightarrow \neg sunny$ is ```True```
- $rain \rightarrow umbrella$ is ```True```
- $(\neg sunny \rightarrow \neg umbrella) \leftrightarrow False$ is ```True```
- $(sunny \vee rain) \rightarrow play$ is ```None```
- $sunny \rightarrow play$ is ```True```
- $\neg rain \leftrightarrow play$ is ```None```
- $\neg play$ is ```None```
- $(sunny \wedge rain \wedge snow) \leftrightarrow False$ is ```True```

#### Example 2 (Group 6-10):
- $rain \leftrightarrow \neg sunny$ is ```True```
- $umbrella \leftrightarrow rain$ is ```True```
- $\neg cloudy \leftrightarrow False$ is ```True```
- $(sunny \vee rain) \rightarrow storm$ is ```None```
- $\neg umbrella \leftrightarrow False$ is ```True```
- $sunny \rightarrow cloudy$ is ```True```
- $(rain \wedge cloudy) \rightarrow storm$ is ```None```
- $(rain \vee cloudy \vee storm)$ is ```True```

#### Example 3 (Group 11-15):
- $windy \leftrightarrow \neg humid$ is ```True```
- $power \rightarrow alert$ is ```False```
- $(\neg humid \rightarrow \neg power) \leftrightarrow False$ is ```True```
- $(windy \vee humid) \rightarrow generator$ is ```None```
- $alert \rightarrow windy$ is ```True```
- $(\neg alert) \leftrightarrow False$ is ```False```
- $\neg generator$ is ```None```
- $(windy \wedge humid \wedge alert) \leftrightarrow False$ is ```True```

#### Example 4 (Group 16-20):
- $traffic \leftrightarrow \neg detour$ is ```True```
- $traffic \rightarrow police$ is ```True```
- $(\neg detour \rightarrow \neg police) \leftrightarrow True$ is ```False```
- $(detour \vee traffic) \rightarrow witness$ is ```None```
- $police \rightarrow report$ is ```True```
- $(\neg report) \leftrightarrow True$ is ```False```
- $\neg witness$ is ```None```
- $(detour \wedge traffic \wedge report) \leftrightarrow False$ is ```True```

#### Example 5 (Group 21-25):
- $engine \leftrightarrow \neg battery$ is ```True```
- $starter \rightarrow engine$ is ```True```
- $(\neg battery \rightarrow \neg starter) \leftrightarrow True$ is ```False```
- $(engine \wedge fuel) \rightarrow starter$ is ```True```
- $battery \rightarrow alarm$ is ```True```
- $\neg engine \leftrightarrow alarm$ is ```None```
- $\neg starter$ is ```False```
- $(engine \wedge starter \wedge battery) \leftrightarrow False$ is ```True```

#### Example 6 (Group 26-30):
- $power \leftrightarrow \neg door$ is ```True```
- $alarm \rightarrow motion$ is ```True```
- $(\neg door \rightarrow \neg alarm) \leftrightarrow False$ is ```True```
- $(motion \vee door) \rightarrow camera$ is ```None```
- $power \rightarrow camera$ is ```None```
- $(\neg door) \leftrightarrow True$ is ```True```
- $\neg camera$ is ```None```
- $(door \wedge alarm \wedge power) \leftrightarrow False$ is ```True```

#### Example 7 (Group 31-35):
- $stocked \leftrightarrow \neg shipment$ is ```True```
- $crew \rightarrow stocked$ is ```True```
- $(\neg shipment \rightarrow crew) \leftrightarrow True$ is ```True```
- $(shipment \vee crew) \rightarrow inspection$ is ```None```
- $delay \rightarrow inspection$ is ```True```
- $(\neg delay) \leftrightarrow True$ is ```True```
- $\neg inspection$ is ```None```
- $(stocked \wedge crew \wedge backup) \leftrightarrow False$ is ```True```

#### Example 8 (Group 36-40):
- $hypothesis \leftrightarrow \neg peer\_review$ is ```True```
- $hypothesis \rightarrow approval$ is ```True```
- $(\neg peer\_review \rightarrow \neg approval) \leftrightarrow False$ is ```True```
- $(evidence \wedge approval) \rightarrow ethics\_clearance$ is ```None```
- $peer\_review \rightarrow ethics\_clearance$ is ```True```
- $\neg approval \leftrightarrow ethics\_clearance$ is ```None```
- $\neg ethics\_clearance$ is ```None```
- $(hypothesis \wedge evidence \wedge funding) \leftrightarrow False$ is ```True```

#### Example 9 (Group 41-45):
- $\neg weather\_clear \leftrightarrow \neg launch$ is ```True```
- $fuel \rightarrow crew\_ready$ is ```True```
- $(\neg weather\_clear \rightarrow \neg fuel) \leftrightarrow False$ is ```True```
- $(launch \vee fuel) \rightarrow abort\_signal$ is ```None```
- $crew\_ready \rightarrow abort\_signal$ is ```None```
- $\neg crew\_ready \leftrightarrow abort\_signal$ is ```None```
- $fuel$ is ```True```
- $(launch \wedge fuel \wedge crew\_ready) \leftrightarrow False$ is ```True```

#### Example 10 (Group 46+):
- $temperature\_high$ is ```True```
- $ac\_on \rightarrow temperature\_high$ is ```True```
- $(\neg windows\_open \rightarrow \neg ac\_on) \leftrightarrow False$ is ```True```
- $(temperature\_high \wedge humidity\_high) \rightarrow maintenance\_mode$ is ```None```
- $windows\_open \rightarrow maintenance\_mode$ is ```True```
- $\neg temperature\_high \leftrightarrow maintenance\_mode$ is ```None```
- $\neg lights\_on$ is ```True```
- $(temperature\_high \wedge humidity\_high \wedge lights\_on) \leftrightarrow False$ is ```True```

Choose valid test model below depending on your assigned example

In [119]:
test_model = {'rain': True,'sunny': False,'umbrella': True,'snow': False,} # Example 1
# test_model = {'rain': True,'sunny': False,'umbrella': True,'cloudy': True,} # Example 2
# test_model = {'windy': True,'humid': False,'power': True,'alert': False,} # Example 3
# test_model = {'detour': False,'accident': True,'traffic': True,'police': True,'report': True,} # Example 4
# test_model = {'engine': True,'battery': False,'fuel': True,'starter': True,} # Example 5
# test_model = {'door': False,'motion': True,'alarm': True, 'power': True,} # Example 6
# test_model = {'stocked': True,'shipment': False,'crew': True,'delay': False,'backup': False,} # Example 7
# test_model = {'hypothesis': True,'evidence': True,'peer_review': False,'approval': True, 'funding': False,} # Example 8
# test_model = { 'launch': False,'fuel': True,'crew_ready': True, 'weather_clear': False,} # Example 9
# test_model = {'temperature_high': True, 'ac_on': True,'windows_open': False,'humidity_high': True,'lights_on': False,} # Example 10

sentence =  Equals(Not("temperature_high"), "maintenance_mode")

#sentence = "temperature_high"
#sentence = Implies("ac_on","temperature_high")
#sentence =  Equals(Implies(Not("windows_open"), Not("ac_on")),False)
#sentence = Implies(And("temperature_high", "humidity_high"),"maintenance_mode")
#sentence1 = Implies("windows_open", "maintenance_mode")
#sentence2 = Equals(Not("temperature_high"), "maintenance_mode")
#sentence4 = Equals(And("temperature_high", "humidity_high", "lights_on"),False)
#sentence3 = Not("lights_on")

S1 ="temperature_high"
S2 ="humidity_high"
S3 ="windows_open"
print(full_sentence_data(Or,[S1,S2,S3]))

print(evaluate(sentence, test_model))


(temperature_high ∨ (humidity_high ∨ windows_open))
None


### Querying for possible models from a Knowledge Base

Task 7: Now we are ready to create a knowledge base.
Making a knowledge base is not that difficult, it's simply a collection of logical sentences that are known (or presumed) to be true in a given environment.

To create this collection we will use the Python ```set``` data structure.
The reason is that sets can only contain one of each element so we don't risk adding the same sentence multiple times.
Sets can be created by listing  a number of elements in curly brackets (```{}```), and elements can be added and removed with the ```.add()``` and ```.remove()``` functions.

The more difficult part is then creating a function for querying the knowledge base for whether a new sentence is true or not.
One simple way to do this query is to test every possible model (every possible combination of truth values for the variables).
For each model we figure out whether the query sentence is True or False, we then also check that all sentences in the knowledge base are True for that model.
Since we "know" that each sentence in the knowledge base should be true, any model that doesn't align with this can be discarded as an "invalid" model of our environment.
For all "valid" models we then return whether that model is True or False.

Of course many environments have a huge amount of variables which would then require us to check a huge number of models.
If we have $n$ variables in the environment we would need to check each sentence $n^2$ times.
However,  not every variable is necessarily relevant for our query sentence.

Using our knowledge of the environment we might be able to deduce the minimum number of variables that are relevant for our query and only test all combinations of them.
This is why we implemented the concept of unknown (```None```) values earlier.
Due to this we also consider a model "valid" if sentence is the knowledge base evaluate as unknown.
This comes with a risk of course, if we mistakenly exclude a relevant variable in our query a model that should be "invalid" might be considered "valid" since a False evaluation might now be unknown.
Additionally, we need to return all "valid" models that are unknown.

We can then use this query function to figure things about the environment.
If a query sentence only gives "valid" models in which it is True, then we know the sentence is True in the environment.
Conversely, if all returned models are False, the sentence is False in the environment.
If it returns some True models and some False models we know that, given what we know of the environment we cannot yet tell whether the sentence is True or not (in some advanced scenarios we might be able to use the number of True/False models to estimate the likelihood of the query being True/False in the actual environment).
If some of the returned models are unknown models, we know that there are relevant variables that were not considered in the query.
This likely means we have an error in the code that determines which variables should be relevant for a query.
Note that many variables that are not in the query sentence could still be relevant.

Finally, if the function returns no valid models at all, this means that there are contradictions in the knowledge base and something has gone wrong in its construction.

Below you will find the skeleton of the query function for you to complete.

In [120]:
import itertools
# Hint: You can use the ```itertools.product()``` function to generate a list of all possible combinations of a list
# [1,2,3]*3 creates a list where the elements of list is repeated 3 times [1,2,3,1,2,3,1,2,3]
# function(*[1,2,3]) is the same as function(1, 2, 3)
list(itertools.product(*[(1,2,3)]*3))

[(1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 3),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3),
 (3, 3, 1),
 (3, 3, 2),
 (3, 3, 3)]

In [121]:
def query(sentence, KB, variables):
  true_models = []
  false_models = []
  none_models = []
  for values in itertools.product([True, False], repeat=len(variables)):
    model = dict(zip(variables, values))
    valid = True
    for s in KB:
     val = evaluate(s, model)
     if val is False:
        valid = False
        break
    if not valid:
        continue
    q_val = evaluate(sentence, model)
    if q_val is True:
        true_models.append(model)
    elif q_val is False:
        false_models.append(model)
    else:
        none_models.append(model)

  return true_models, false_models, none_models

### Testing the querying function

No complex function is complete without testing.
To do this we will use the sentences from testing our evaluation function as the knowledge base.
You will be given a list of "relevant" variables and then you will query the knowledge base with the following sentences with the given expected results.

The query function should be designed after the design requirments stated (arg/return type).

The solution is accepted only if the querying function exactly returns the stated expected result of each query.

#### Example 1 (Group 1-5)
- $True$ returns 2 True models (and 0 False/None models).
- $rain \leftrightarrow \neg sunny $ returns 2 True models.
- $play \wedge cloudy$ returns 1 False model, and 1 unknown model.
- $sunny \leftrightarrow play$ returns 2 unknown models.
- $(sunny \leftrightarrow \neg cloudy) \leftrightarrow False$ returns 1 True and 1 False model.

#### Example 2 (Group 6-10)
- $True$ returns 1 True model (and 0 False/None models).
- $rain \leftrightarrow \neg sunny$ returns 1 True model.
- $storm \wedge cloudy$ returns 1 unknown model.
- $sunny \leftrightarrow storm$ returns 1 unknown model
- $(sunny \leftrightarrow \neg cloudy) \leftrightarrow False$ returns 1 False model.

#### Example 3 (Group 10-15)
- $True$ returns 1 True model (and 0 False/None models).
- $windy \leftrightarrow \neg humid$ returns 1 True model.
- $power \wedge generator$ returns 1 unknown model.
- $alert \leftrightarrow generator$ returns 1 unknown model.
- $(alert \leftrightarrow \neg humid) \leftrightarrow False$ returns 1 False model.

#### Example 4 (Group 16-20)
- $True$ returns 2 True models (and 0 False/None models).
- $traffic \leftrightarrow \neg detour$ returns 2 True models (and 0 False/None models).
- $police \wedge report$ returns 2 False models.
- $police \leftrightarrow report$ returns 2 True models.
- $(witness \leftrightarrow \neg report) \leftrightarrow False$ returns 2 unknown models

#### Example 5 (Group 21-25)
- $True$ returns 3 True models (and 0 False/None models).
- $engine \leftrightarrow \neg battery$ returns 3 True models.
- $engine \wedge starter$ returns 3 False models.
- $starter \leftrightarrow alarm$ returns 3 unknown models
- $(starter \leftrightarrow \neg fuel) \leftrightarrow False$ returns 2 True models and 1 False model.

#### Example 6 (Group 26-30)
- $True$ returns 1 True model (and 0 False/None models).
- $power \leftrightarrow \neg door$ returns 1 True model.
- $alarm \wedge camera$ returns 1 unknown model.
- $power \leftrightarrow camera$ returns 1 unknown model.
- $(alarm \leftrightarrow \neg motion) \leftrightarrow False$ returns 1 True model.

#### Example 7 (Group 31-35)
- $True$ returns 3 True models (and 0 False/None models).
- $stocked \leftrightarrow \neg shipment$ returns 3 True models.
- $crew \wedge backup$ returns 3 False models.
- $crew \leftrightarrow stocked$ 3 True models
- $(inspection \leftrightarrow \neg delay) \leftrightarrow False$ returns 3 unknown models.

#### Example 8 (Group 36-40)
- $True$ returns 3 True models (and 0 False/None models).
- $hypothesis \leftrightarrow \neg evidence$ returns 2 True models and 1 False model.
- $approval \wedge peer\_review$ returns 3 False models.
- $evidence \leftrightarrow approval$ returns 1 True model and 2 False models.
- $(evidence \leftrightarrow \neg peer\_review) \leftrightarrow False$ returns 2 True models and 1 False model.

#### Example 9 (Group 41-45)
- $False$ returns 1 False model (and 0 False/None models).
- $launch \leftrightarrow \neg weather\_clear$ returns 1 False model.
- $crew\_ready \wedge abort\_signal$ returns 1 unknown model.
- $abort\_signal \leftrightarrow crew\_ready$ returns 1 unknown model.
- $(abort\_signal \leftrightarrow \neg weather\_clear) \leftrightarrow False$ returns 1 unknown model.

#### Example 10 (Group 46-50+)
- $True$ returns 2 True models (and 0 False/None models).
- $temperature\_high \leftrightarrow \neg ac\_on$ returns 2 False models.
- $lights\_on \wedge humidity\_high$ returns 2 False models.
- $ac\_on \leftrightarrow lights\_on$ returns 2 False models.
- $(ac\_on \leftrightarrow \neg humidity\_high) \leftrightarrow False$ returns 1 True model and 1 False model.

Below you have test variables. Select test variables based on your assigned example.

In [122]:
test_variables = {"rain", "umbrella", "cloudy", "sunny"} # Example 1
# test_variables = {"rain", "sunny", "umbrella", "cloudy"} # Example 2
# test_variables = {"windy", "humid", "power", "alert"} # Example 3
# test_variables = {"detour", "accident", "traffic", "police", "report"} # Example 4
# test_variables = {"engine", "battery", "fuel", "starter"} # Example 5
# test_variables = {"door", "motion", "alarm", "power"} # Example 6
# test_variables = {"stocked", "shipment", "crew", "delay", "backup"} # Example 7
# test_variables = test_variables = {"hypothesis", "evidence", "peer_review", "approval", "funding"} # Example 8
# test_variables = {"launch", "fuel", "crew_ready", "weather_clear"} # Example 9
# test_variables = {"temperature_high", "ac_on", "windows_open", "humidity_high", "lights_on"} # Example 10

KB = {
    Equals("rain", Not("sunny")),
    Implies("rain", "umbrella"),
    Equals(Implies(Not("sunny"), Not("umbrella")), False),
    Implies(Or("sunny", "rain"), "play"),
    Implies("sunny", "play"),
    Equals(Not("rain"), "play"),
    Not("play"),
    Equals(And("sunny", "rain", "snow"), False)

}   #Add all the sentences that were used to test evaluation previously to the knowledge base


query_result = query(Equals(Equals("sunny", Not("cloudy")), False), KB, test_variables)
print(query_result)

#

 #Add all the sentences that were used to test evaluation previously to the knowledge base



([{'sunny': False, 'rain': True, 'umbrella': True, 'cloudy': False}], [{'sunny': False, 'rain': True, 'umbrella': True, 'cloudy': True}], [])


### Before submitting to Canvas

Important! Make sure you follow design requirements for the evaluate and query function (correct argument- and return type). The notebook will be unit tested and verified.

